In [ ]:
import pandas as pd
from utils import download_data
from data_preparation import data_preparation, train_val_test_split

In [ ]:
data = download_data(user='cchallu', password='XXX',
                     tb_name='sketch.train_data')

In [ ]:
data_final = data_preparation(data, args=1)

In [ ]:
train_data, validation_data, test_data = train_val_test_split(data_final)